#Introducción

A principios de este año, junto con otra persona con quien compartimos ideas y pensamos proyectos para desarrollar, comenzamos a planear un sistema mejorado que se encargue de controlar y modificar el funcionamiento de los semáforos de una ciudad. 

Esta idea se encuentra en sus primeros pasos y surge ante la necesidad de eliminar parcialmente momentos de espera innecesarios en semáforos rojos, como también mejorar la circulación en áreas y horarios de atascos y prevenir choques en cadena, por ejemplo. Creemos que en determinadas circunstancias como estas el proyecto puede llegar a ser provechoso si termina concretándose.

Presentado el caso, parece buena oportunidad aprovechar este curso para analizar los accidentes viales en una ciudad, y aislar los casos más relevantes para el proyecto, aquellos que a través de una mejora en el sistema pueden erradicarse o disminuir el número de víctimas.

#Import de librerías

[Carpeta de Google Drive](https://drive.google.com/drive/folders/1MSHoIXMT33QFXHwsy_IxdxQFE2tL95gB?usp=share_link)



In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.notebook_repr_html',True)
pd.set_option('display.max_columns', 30) 

%matplotlib inline

from google.colab import drive
import os
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [57]:
%cd /content/gdrive/MyDrive/Data Science
print (' ')
%ls

/content/gdrive/MyDrive/Data Science
 
barcelona_traffic_accidents.csv  inventario_semaforos.csv
causas_accidentes.csv            inventario_sendas_peatonales.csv
Datasets_Vasquez.ipynb           netflix_movies.csv
info_accidentes.csv


#Tráfico en Barcelona

Luego de buscar en distintas plataformas, logré encontrar potenciales datasets que luego de trabajarlos, pueda obtener insights y resultados que me guíen por dónde encarar el proyecto personal.


###[Open Data BCN](https://opendata-ajuntament.barcelona.cat/es/)

Si bien hubiese preferido conseguir datasets del transito de alguna ciudad argentina, pude encontrar muchísima más información acerca de la ciudad de Barcelona, la cuál me permitirá trabajar mejor. Open Data BCN es un servicio de datos abiertos del Ayuntamiento de Barcelona. Cuenta con un catálogo de datasets amplio y variado.

#Dataset #1
##Accidentes de Tránsito en Barcelona

Información de todos los accidentes de tránsito en Barcelona a lo largo de los años. Este dataset puede servir para analizar la variación a lo largo de los distintos años, y ver qué dato surge como constante a través del tiempo. 

Las variables a tener más en cuenta son las locaciones (ya sea por barrio,departamento o coordenadas especificas de una esquina), y el momento del accidente (hora, día). En relación a estas vería la cantidad promedio de víctimas por época del año y por zona.

In [ ]:
!head -10 barcelona_traffic_accidents.csv
df_barcelona_traffic_accidents = pd.read_csv('barcelona_traffic_accidents.csv',sep = ',')

In [ ]:
df_barcelona_traffic_accidents[['case_id','district_id','district_name','neighborhood_id','neighborhood_name','street_code','street_name','postal_code','weekday_name','weekday','type_day','year','month','month_name','day','hour','cyrcadian','cause_incident','n_deaths','n_wounded_mild','n_wounded_severe','n_victims','n_vehicles','utm_coordinate_y','utm_coordinate_x','longitude','latitude']]

In [60]:
df_barcelona_traffic_accidents.shape # 110655 filas y 27 columnas

(110655, 27)

In [61]:
df_barcelona_traffic_accidents = df_barcelona_traffic_accidents.rename(columns = {'case_id':'Case ID', 'district_id':'District ID', 'district_name':'District Name', 'neighborhood_id':'Neighborhood ID', 'neighborhood_name':'Neighborhood Name', 'street_code':'Street ID', 'street_name': 'Street Name', 'postal_code':'Postal Code', 'weekday_name':'Weekday Name', 'weekday':'Weekday', 'type_day':'Type Day', 'year':'Year', 'month':'Month', 'month_name':'Month Name', 'day':'Day', 'hour':'Hour', 'cyrcadian':'Time of Day', 'cause_incident':'Cause', 'n_deaths':'N° Deaths', 'n_wounded_mild':'N° Wounded Mild', 'n_wounded_severe':'N° Wounded Severe', 'n_victims':'N° Victims', 'n_vehicles':'N° Vehicles', 'utm_coordinate_x':'Coordinate X', 'utm_coordinate_y':'Coordinate Y', 'longitude':'Longitude', 'latitude':'Latitude'})
df_barcelona_traffic_accidents.head() # rename para un mejor entendimiento de las variables

,Case ID,District ID,District Name,Neighborhood ID,Neighborhood Name,Street ID,Street Name,Postal Code,Weekday Name,Weekday,Type Day,Year,Month,Month Name,Day,Hour,Time of Day,Cause,N° Deaths,N° Wounded Mild,N° Wounded Severe,N° Victims,N° Vehicles,Coordinate Y,Coordinate X,Longitude,Latitude
0,2015S005807,-1.0,Desconegut,-1.0,Desconegut,-1.0,Desconegut,Desconegut,Dimarts,Dm,Laboral,2015.0,7.0,Juliol,28.0,12.0,Matí,No és causa del vianant,0.0,2.0,0.0,2.0,2.0,-1,-1,NaN,NaN
1,2015S007685,10.0,Sant Martí,64.0,el Camp de l'Arpa del Clot,134801.0,Freser,0208 0208,Dimarts,Dm,Laboral,2015.0,10.0,Octubre,13.0,19.0,Tarda,Desobeir el senyal del semàfor,0.0,1.0,0.0,1.0,1.0,"4585420,58","431779,16",NaN,NaN
2,2015S001364,10.0,Sant Martí,64.0,el Camp de l'Arpa del Clot,161407.0,Indústria,0336 0336,Dissabte,Ds,Laboral,2015.0,2.0,Febrer,21.0,21.0,Tarda,No és causa del vianant,0.0,1.0,0.0,1.0,1.0,"4585555,86","431913,65",NaN,NaN
3,2015S004325,10.0,Sant Martí,64.0,el Camp de l'Arpa del Clot,226400.0,Las Navas de Tolosa,0343 0343,Divendres,Dv,Laboral,2015.0,6.0,Juny,5.0,9.0,Matí,No és causa del vianant,0.0,1.0,0.0,1.0,2.0,"4585565,44","431946,45",NaN,NaN
4,2015S005540,10.0,Sant Martí,64.0,el Camp de l'Arpa del Clot,95506.0,Conca,0032 0034,Divendres,Dv,Laboral,2015.0,7.0,Juliol,17.0,20.0,Tarda,No és causa del vianant,0.0,2.0,0.0,2.0,1.0,"4585260,16","431530,84",NaN,NaN


In [62]:
df_barcelona_traffic_accidents.columns # variables ya renombradas del dataset

Index(['Case ID', 'District ID', 'District Name', 'Neighborhood ID',
       'Neighborhood Name', 'Street ID', 'Street Name', 'Postal Code',
       'Weekday Name', 'Weekday', 'Type Day', 'Year', 'Month', 'Month Name',
       'Day', 'Hour', 'Time of Day', 'Cause', 'N° Deaths', 'N° Wounded Mild',
       'N° Wounded Severe', 'N° Victims', 'N° Vehicles', 'Coordinate Y',
       'Coordinate X', 'Longitude', 'Latitude'],
      dtype='object')

In [63]:
df_barcelona_traffic_accidents.info() #resumen del dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110655 entries, 0 to 110654
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Case ID            110655 non-null  object 
 1   District ID        110654 non-null  float64
 2   District Name      110654 non-null  object 
 3   Neighborhood ID    110654 non-null  object 
 4   Neighborhood Name  110654 non-null  object 
 5   Street ID          110654 non-null  float64
 6   Street Name        110653 non-null  object 
 7   Postal Code        105851 non-null  object 
 8   Weekday Name       110655 non-null  object 
 9   Weekday            102995 non-null  object 
 10  Type Day           102995 non-null  object 
 11  Year               110654 non-null  float64
 12  Month              110654 non-null  float64
 13  Month Name         110654 non-null  object 
 14  Day                110654 non-null  float64
 15  Hour               110654 non-null  float64
 16  Ti

In [64]:
df_barcelona_traffic_accidents.describe()

,District ID,Street ID,Year,Month,Day,Hour,N° Deaths,N° Wounded Mild,N° Wounded Severe,N° Victims,N° Vehicles,Longitude,Latitude
count,110654.000000,110654.000000,110654.000000,110654.000000,110654.000000,110654.000000,110654.000000,110654.000000,110654.000000,110654.000000,110654.000000,54345.000000,54345.000000
mean,4.813563,238669.737886,2015.371166,6.540089,15.759765,13.819302,0.002612,1.158964,0.023695,1.185271,1.915023,3.350592,60.405022
std,2.988493,190947.366687,3.316393,3.476227,8.748838,5.289038,0.052780,0.772026,0.162776,0.766085,0.729919,50.594181,886.553616
min,-1.000000,-1.000000,2010.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.488576,0.000009
25%,2.000000,112900.000000,2013.000000,3.250000,8.000000,10.000000,0.000000,1.000000,0.000000,1.000000,2.000000,2.145108,41.385786
50%,4.000000,187207.000000,2015.000000,6.000000,16.000000,14.000000,0.000000,1.000000,0.000000,1.000000,2.000000,2.163498,41.397152
75%,7.000000,312400.000000,2018.000000,10.000000,23.000000,18.000000,0.000000,1.000000,0.000000,1.000000,2.000000,2.180154,41.411468
max,10.000000,701869.000000,2021.000000,12.000000,31.000000,23.000000,3.000000,25.000000,4.000000,26.000000,18.000000,2189.000000,41414.000000


#Dataset #2A
##Consecuencias de accidentes

Este dataset muestra las personas involucradas en un accidente gestionado por la Guardia Urbana en la ciudad de Barcelona que han sufrido algún tipo de lesión (herido leve, herido grave o muerte). Incluye la descripción de la persona (conductor, pasajero o peatón), sexo, edad, vehículo asociado a la persona y si la causa ha sido del peatón.

Este dataset sólo corresponde al año 2021, por lo que el análisis no se hará a lo largo de los años, sino por época de un mismo año.

Lo interesante de esta información es que se adentra más en cada persona, y podemos obtener data acerca de qué clase de peatón o conductor es más propenso a estar involucrado en un accidente, y quién fue el culpable en cada caso.

In [ ]:
!head -10 df_info_accidentes.csv
df_info_accidentes = pd.read_csv('info_accidentes.csv',sep = ',') # importamos un csv y lo convertimos a df

In [ ]:
df_info_accidentes[['Numero_Expedient','Codi_districte','Nom_districte','Codi_barri','Nom_barri','Codi_carrer','Nom_carrer','Num_postal','Descripcio_dia_setmana','NK_ Any','Mes_ any','Nom_mes','Dia_mes','Descripcio_torn','Hora_dia','Descripcio_causa_vianant','Desc_Tipus_vehicle_implicat','Descripció_sexe','Edat','Descripció_tipus_persona','Descripcio_Lloc_atropellament_vianat','Descripcio_Motiu_desplaÃ§ament_vianant','Descripcio_Motiu_desplaÃ§ament_conductor','Descripcio_victimitzacio','Coordenada_UTM_X_ED50','Coordenada_UTM_Y_ED50','Longitud_WGS84','Latitud_WGS84']]

In [67]:
df_info_accidentes.shape # 8854 filas y 28 columnas

(8854, 28)

In [68]:
df_info_accidentes = df_info_accidentes.rename(columns ={'Numero_Expedient':'Case ID', 'Codi_districte':'District ID', 'Nom_districte':'District Name', 'Codi_barri':'Neighborhood ID', 'Nom_barri':'Neighborhood Name', 'Codi_carrer':'Street ID', 'Nom_carrer':'Street Name', 'Num_postal':'Postal Code', 'Descripcio_dia_setmana':'Weekday', 'NK_ Any':'Year', 'Mes_ any':'Month', 'Nom_mes':'Month Name', 'Dia_mes':'Day', 'Descripcio_torn':'Time of Day', 'Hora_dia':'Hour', 'Descripcio_causa_vianant':'Cause', 'Desc_Tipus_vehicle_implicat':'Vehicle Involved', 'Descripció_sexe':'Sex', 'Edat':'Edad', 'Descripció_tipus_persona':'Person','Descripcio_Lloc_atropellament_vianat':'Place of hit','Descripcio_Motiu_desplaÃ§ament_vianant':'Reason of Pedestrian', 'Descripcio_Motiu_desplaÃ§ament_conductor':'Reason of Driver','Descripcio_victimitzacio':'Description of Victim', 'Coordenada_UTM_X_ED50':'Coordenate X', 'Coordenada_UTM_Y_ED50':'Coordenate Y', 'Longitud_WGS84':'Longitude', 'Latitud_WGS84':'Latitude'})
df_info_accidentes.head() # rename para un mejor entendimiento de las variables

,Case ID,District ID,District Name,Neighborhood ID,Neighborhood Name,Street ID,Street Name,Postal Code,Weekday,Year,Month,Month Name,Day,Time of Day,Hour,Cause,Vehicle Involved,Sex,Edad,Person,Place of hit,Reason of Pedestrian,Reason of Driver,Description of Victim,Coordenate X,Coordenate Y,Longitude,Latitude
0,2021S001906,1,Ciutat Vella,1,el Raval,68503,Carretes,0044 0044,Dimarts,2021,4,Abril,13,Tarda,20,No és causa del vianant,Bicicleta,Dona,8,Vianant,A la vorera / Andana ...,Oci i entreteniment,Desconegut,Ferit lleu: Hospitalització fins a 24h,430455.95,4581207.96,2.167209,41.377541
1,2021S001254,1,Ciutat Vella,1,el Raval,198804,Paral·lel,0050B0050B,Divendres,2021,3,Març,12,Matí,12,No és causa del vianant,Taxi,Home,58,Vianant,A la vorera / Andana ...,Oci i entreteniment,Desconegut,Ferit lleu: Hospitalització fins a 24h,430812.07,4580929.31,2.171499,41.375062
2,2021S005693,1,Ciutat Vella,1,el Raval,250904,Pelai,0056 0056,Dilluns,2021,10,Octubre,11,Tarda,19,No és causa del vianant,Veh. mobilitat personal amb motor,Dona,52,Vianant,A la vorera / Andana ...,Desconegut,Desconegut,Ferit lleu: Hospitalització fins a 24h,430626.85,4582082.01,2.169152,41.385428
3,2021S000623,1,Ciutat Vella,1,el Raval,282106,Riera Alta,0010 0010,Dissabte,2021,2,Febrer,6,Tarda,15,No és causa del vianant,Bicicleta,Home,47,Vianant,A la vorera / Andana ...,Desconegut,Desconegut,Ferit lleu: Rebutja assistència sanitària,430434.78,4581547.79,2.166916,41.380600
4,2021S005042,1,Ciutat Vella,1,el Raval,701367,Raval,0053 0053,Diumenge,2021,9,Setembre,12,Tarda,17,No és causa del vianant,Veh. mobilitat personal amb motor,Home,4,Vianant,A la vorera / Andana ...,Desconegut,Desconegut,Ferit lleu: Hospitalització fins a 24h,430746.05,4581254.77,2.170672,41.377988


In [ ]:
df_info_accidentes.columns # variables ya renombradas del dataset

Index(['Case ID', 'District ID', 'District Name', 'Neighborhood ID',
       'Neighborhood Name', 'Street ID', 'Street Name', 'Postal Code',
       'Weekday', 'Year', 'Month', 'Month Name', 'Day', 'Time of Day', 'Hour',
       'Cause', 'Vehicle Involved', 'Sex', 'Edad', 'Person', 'Place of hit',
       'Reason of Pedestrian', 'Reason of Driver', 'Description of Victim',
       'Coordenate X', 'Coordenate Y', 'Longitude', 'Latitude'],
      dtype='object')

In [ ]:
df_info_accidentes.info() # resumen del dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8854 entries, 0 to 8853
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Case ID                8854 non-null   object 
 1   District ID            8854 non-null   int64  
 2   District Name          8854 non-null   object 
 3   Neighborhood ID        8854 non-null   int64  
 4   Neighborhood Name      8854 non-null   object 
 5   Street ID              8854 non-null   int64  
 6   Street Name            8854 non-null   object 
 7   Postal Code            8854 non-null   object 
 8   Weekday                8854 non-null   object 
 9   Year                   8854 non-null   int64  
 10  Month                  8854 non-null   int64  
 11  Month Name             8854 non-null   object 
 12  Day                    8854 non-null   int64  
 13  Time of Day            8854 non-null   object 
 14  Hour                   8854 non-null   int64  
 15  Caus

In [ ]:
df_info_accidentes.describe()

,District ID,Neighborhood ID,Street ID,Year,Month,Day,Hour,Coordenate X,Coordenate Y,Longitude,Latitude
count,8854.000000,8854.000000,8854.000000,8854.0,8854.000000,8854.000000,8854.000000,8854.000000,8.854000e+03,8825.000000,8825.000000
mean,4.930540,27.724870,253657.392591,2021.0,6.797267,15.600858,14.169867,428725.843543,4.568742e+06,2.163077,41.400454
std,3.022381,22.198899,206200.952440,0.0,3.416478,8.648562,4.991267,24662.770882,2.619281e+05,0.024357,0.022266
min,-1.000000,-1.000000,-1.000000,2021.0,1.000000,1.000000,0.000000,-1.000000,-1.000000e+00,2.064405,41.322721
25%,2.000000,8.000000,115603.000000,2021.0,4.000000,8.000000,11.000000,428631.845000,4.582128e+06,2.145360,41.385958
50%,4.000000,21.000000,191204.000000,2021.0,7.000000,16.000000,14.000000,430172.550000,4.583481e+06,2.163632,41.397964
75%,8.000000,44.000000,326179.750000,2021.0,10.000000,23.000000,18.000000,431625.870000,4.585130e+06,2.180776,41.412942
max,10.000000,73.000000,701857.000000,2021.0,12.000000,31.000000,23.000000,435119.930000,4.590733e+06,2.222571,41.463418


#Dataset #2B
##Causas de accidentes

Este dataset consiste en un listado de  de la causalidad de los accidentes gestionados por la Guardia Urbana en la ciudad de Barcelona. Un accidente puede tener una o más causas mediatas las cuales hacen referencia a factores externos del resultado en tiempo, lugar o grado (ej : Alcoholemia y Exceso de velocidad o inadecuada).

Al igual que el dataset #2, este corresponde al año 2021 y se comporta como un complemento del anterior dataset, centrándose en las causas más que en los resultados del incidente.

Es el dataset más importante, ya que la diferencia respecto a los anteriores es que posee información de los motivos del incidente. De esta manera podremos analizar qué accidentes pueden ser evitados y cuáles no dependen del mejoramiento del sistema de semáforos.

In [ ]:
!head -10 causas_accidentes.csv
df_causas_accidentes = pd.read_csv('causas_accidentes.csv',sep = ',') # importamos un csv y lo convertimos a df

In [ ]:
df_causas_accidentes[['Numero_expedient','Codi_districte','Nom_districte','Codi_barri','Nom_barri','Codi_carrer','Nom_carrer','Num_postal ','Descripcio_dia_setmana','NK_Any','Mes_any','Nom_mes','Dia_mes','Descripcio_torn','Hora_dia','Descripcio_causa_mediata','Coordenada_UTM_X_ED50','Coordenada_UTM_Y_ED50','Longitud_WGS84','Latitud_WGS84']]

In [ ]:
df_causas_accidentes.shape # 7653 filas y 20 columnas

(7653, 20)

In [ ]:
df_causas_accidentes = df_causas_accidentes.rename( columns = {'Numero_expedient':'Case ID', 'Codi_districte':'District ID', 'Nom_districte':'District Name', 'Codi_barri':'Neighborhood ID', 'Nom_barri':'Neighborhood Name', 'Codi_carrer':'Street ID', 'Nom_carrer':'Street Name', 'Num_postal ':'Postal Code', 'Descripcio_dia_setmana':'Weekday', 'NK_Any':'Year', 'Mes_any':'Month', 'Nom_mes':'Month Name', 'Dia_mes':'Day', 'Descripcio_torn':'Cause', 'Hora_dia':'Hour', 'Descripcio_causa_mediata':'Mediate Cause', 'Coordenada_UTM_X_ED50':'Coordante X', 'Coordenada_UTM_Y_ED50':'Coordante Y', 'Longitud_WGS84':'Longitude', 'Latitud_WGS84':'Latitude'})
df_causas_accidentes.head() # rename para un mejor entendimiento de las variables

,Case ID,District ID,District Name,Neighborhood ID,Neighborhood Name,Street ID,Street Name,Postal Code,Weekday,Year,Month,Month Name,Day,Hour,Cause,Mediate Cause,Coordante X,Coordante Y,Longitude,Latitude
0,2021S007098,-1,Desconegut,-1,Desconegut,-1,Número 4 Zona Franca ...,85,Dimarts,2021,12,Desembre,7,6,Matí,No hi ha causa mediata,426925.25,4576176.57,2.125596,41.331912
1,2021S005729,-1,Desconegut,-1,Desconegut,-1,Corts ...,65-71,Divendres,2021,10,Octubre,8,14,Tarda,No hi ha causa mediata,426674.42,4581347.85,2.121974,41.378465
2,2021S006593,-1,Desconegut,-1,Desconegut,-1,Joan XXIII ...,3-15,Dimecres,2021,11,Novembre,17,14,Tarda,No hi ha causa mediata,426623.16,4581757.35,2.121311,41.382148
3,2021S004679,10,Sant Martí,64,el Camp de l'Arpa del Clot,346402,Trinxant,0073 0075,Dimarts,2021,8,Agost,24,16,Tarda,No hi ha causa mediata,431951.08,4585370.32,2.184619,41.415158
4,2021S000960,10,Sant Martí,64,el Camp de l'Arpa del Clot,161407,Indústria,0336 0336,Dimecres,2021,2,Febrer,24,16,Tarda,No hi ha causa mediata,431913.59,4585555.55,2.184149,41.416823


In [ ]:
df_causas_accidentes.columns # variables ya renombradas del dataset

Index(['Numero_expedient', 'Codi_districte', 'Nom_districte', 'Codi_barri',
       'Nom_barri', 'Codi_carrer', 'Nom_carrer', 'Num_postal ',
       'Descripcio_dia_setmana', 'NK_Any', 'Mes_any', 'Nom_mes', 'Dia_mes',
       'Hora_dia', 'Descripcio_torn', 'Descripcio_causa_mediata',
       'Coordenada_UTM_X_ED50', 'Coordenada_UTM_Y_ED50', 'Longitud_WGS84',
       'Latitud_WGS84'],
      dtype='object')

In [ ]:
df_causas_accidentes.info() #resumen del dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7653 entries, 0 to 7652
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Numero_expedient          7653 non-null   object 
 1   Codi_districte            7653 non-null   int64  
 2   Nom_districte             7653 non-null   object 
 3   Codi_barri                7653 non-null   int64  
 4   Nom_barri                 7653 non-null   object 
 5   Codi_carrer               7653 non-null   int64  
 6   Nom_carrer                7653 non-null   object 
 7   Num_postal                7653 non-null   object 
 8   Descripcio_dia_setmana    7653 non-null   object 
 9   NK_Any                    7653 non-null   int64  
 10  Mes_any                   7653 non-null   int64  
 11  Nom_mes                   7653 non-null   object 
 12  Dia_mes                   7653 non-null   int64  
 13  Hora_dia                  7653 non-null   int64  
 14  Descripc

In [ ]:
df_causas_accidentes.describe() 

,Codi_districte,Codi_barri,Codi_carrer,NK_Any,Mes_any,Dia_mes,Hora_dia,Coordenada_UTM_X_ED50,Coordenada_UTM_Y_ED50,Longitud_WGS84,Latitud_WGS84
count,7653.000000,7653.00000,7653.000000,7653.0,7653.000000,7653.000000,7653.000000,7653.000000,7.653000e+03,7630.000000,7630.000000
mean,4.951784,27.84934,247232.313080,2021.0,6.823599,15.691755,14.020515,428842.648463,4.570002e+06,2.163082,41.400659
std,3.016491,22.16014,199602.357093,0.0,3.427996,8.658849,5.135002,23635.128156,2.509385e+05,0.024356,0.022257
min,-1.000000,-1.00000,-1.000000,2021.0,1.000000,1.000000,0.000000,-1.000000,-1.000000e+00,2.064405,41.322721
25%,2.000000,8.00000,114000.000000,2021.0,4.000000,8.000000,11.000000,428632.380000,4.582151e+06,2.145322,41.386117
50%,4.000000,21.00000,191204.000000,2021.0,7.000000,16.000000,14.000000,430181.980000,4.583506e+06,2.163709,41.398251
75%,8.000000,44.00000,320908.000000,2021.0,10.000000,23.000000,18.000000,431621.760000,4.585167e+06,2.180673,41.413370
max,10.000000,73.00000,701857.000000,2021.0,12.000000,31.000000,23.000000,435119.930000,4.590863e+06,2.222571,41.464644


#Dataset Extra
##Inventario de semáforos de Barcelona

Este dataset es un inventario de los elementos de la regulación semafórica de la red de infraestructuras de la ciudad de Barcelona, correspondiente al año 2018. Los elementos sin información de barrio y districto corresponden a ubicaciones limítrofes de la ciudad.

Esta información nos sirve para saber en qué lugares y esquinas va a ser posible implementar el sistema mejorado de semáforos.

In [ ]:
!head -10 inventario_semaforos.csv
df_inventario_semaforos = pd.read_csv('inventario_semaforos.csv',sep = ',') # importamos un csv y lo convertimos a df

In [ ]:
df_inventario_semaforos[['ID_Semafor','Codi_Semafor','Codi_Districte','Nom_Districte','Codi_Barri','Nom_Barri','Longitud','Latitud','X_ETRS89','Y_ETRS89','Data_Alta']] 

In [ ]:
df_inventario_semaforos.shape # 115676 columnas y 11 filas

(115676, 11)

In [ ]:
df_inventario_semaforos = df_inventario_semaforos.rename( columns = {'ID_Semafor':'Traffic Light ID', 'Codi_Semafor':'Traffic Light Code', 'Codi_Districte':'District ID', 'Nom_Districte':'District Name', 'Codi_Barri':'Neighborhood Code', 'Nom_Barri':'Neighborhood Name', 'Longitud':'Longitude', 'Latitud':'Latitude', 'X_ETRS89':'Coordenate X', 'Y_ETRS89':'Coordenate Y', 'Data_Alta':'Release Date'})
df_inventario_semaforos.head() # rename para un mejor entendimiento de las variables

,Traffic Light ID,Traffic Light Code,District ID,District Name,Neighborhood Code,Neighborhood Name,Longitude,Latitude,Coordenate X,Coordenate Y,Release Date
0,415227,ArmarioSim,NaN,NaN,NaN,NaN,2.094896,41.375543,424312.672,4580842.381,1900/01/01
1,415228,ArmarioSim,NaN,NaN,NaN,NaN,2.091653,41.369542,424034.492,4580179.061,1900/01/01
2,415229,ArmarioSim,NaN,NaN,NaN,NaN,2.089232,41.355098,423815.166,4578577.595,1900/01/01
3,415242,ArmarioSim,NaN,NaN,NaN,NaN,2.225559,41.418246,435281.562,4585477.333,1900/01/01
4,415243,ArmarioSim,NaN,NaN,NaN,NaN,2.220889,41.421307,434894.303,4585820.716,1900/01/01


In [ ]:
df_inventario_semaforos.columns # variables ya renombradas del dataset

Index(['Traffic Light ID', 'Traffic Light Code', 'District ID',
       'District Name', 'Neighborhood Code', 'Neighborhood Name', 'Longitude',
       'Latitude', 'Coordenate X', 'Coordenate Y', 'Release Date'],
      dtype='object')

In [ ]:
df_inventario_semaforos.info() #resumen del dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115676 entries, 0 to 115675
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Traffic Light ID    115676 non-null  int64  
 1   Traffic Light Code  115676 non-null  object 
 2   District ID         115529 non-null  float64
 3   District Name       115529 non-null  object 
 4   Neighborhood Code   115529 non-null  float64
 5   Neighborhood Name   115529 non-null  object 
 6   Longitude           115676 non-null  float64
 7   Latitude            115676 non-null  float64
 8   Coordenate X        115676 non-null  float64
 9   Coordenate Y        115676 non-null  float64
 10  Release Date        115676 non-null  object 
dtypes: float64(6), int64(1), object(4)
memory usage: 9.7+ MB


In [ ]:
df_inventario_semaforos.describe()

,Traffic Light ID,District ID,Neighborhood Code,Longitude,Latitude,Coordenate X,Coordenate Y
count,115676.000000,115529.000000,115529.000000,115676.000000,115676.000000,115676.000000,1.156760e+05
mean,436172.491485,5.697349,33.611552,2.167046,41.402827,430374.939935,4.583811e+06
std,60599.192585,3.171651,24.008491,0.025786,0.019791,2164.900820,2.188064e+03
min,309883.000000,1.000000,1.000000,2.088800,41.339963,423785.832000,4.576870e+06
25%,393693.250000,2.000000,10.000000,2.146793,41.388814,428666.208500,4.582261e+06
50%,449158.500000,5.000000,26.000000,2.169517,41.400910,430583.575000,4.583603e+06
75%,485848.250000,9.000000,60.000000,2.186666,41.414610,432021.742000,4.585110e+06
max,520795.000000,10.000000,73.000000,2.225827,41.464078,435303.722000,4.590597e+06


#Dataset #3
##Películas de Netflix

Este dataset fue actualizado hasta Julio de 2022 con información encontrada en Estados Unidos, y muestra el listado de películas disponibles en la plataforma Netflix.

Si bien es el dataset que menos me entusiasma trabjar, me pareció una buena idea poder trabajar con data de distinta temática que los anteriores datasets. Hay una gran variedad de datos y de distintos tipo, y también varios datos null, con los que podré trabajar.

Algunos de los resultados posibles viendo las variables que dispone pueden ser el hallar qué tipo o case de película le da mayor prestigio a la plataforma, y guiar a Netflix en futuro camino posible para que siga siendo una de las plataformas de streaming más utilizadas. Otro insight potencial es buscar directores y actores que generen buenos resultados en la audiencia, para contratarlos para próximas producciones propias de Netflix.


In [ ]:
!head -10 netflix_movies.csv
df_netflix_movies = pd.read_csv('netflix_movies.csv',sep = ',')

In [ ]:
df_netflix_movies[['id','title','type','description','release_year','age_certification','runtime','genres','production_countries','seasons','imdb_id','imdb_score','imdb_votes','tmdb_popularity','tmdb_score']]

In [ ]:
df_netflix_movies.shape # 5850 filas y 15 columnas

(5850, 15)

In [ ]:
df_netflix_movies = df_netflix_movies.rename( columns = {'id':'ID', 'title':'Title', 'type':'Type', 'description':'Description', 'release_year':'Release Year', 'age_certification':'Age Certification', 'runtime':'Runtime', 'genres':'Genres', 'production_countries':'Production Countries', 'seasons':'Seasons', 'imdb_id':'IMDB ID', 'imdb_score':'IMDB Score', 'imdb_votes':'IMDB Votes', 'tmdb_popularity':'TMDB Popularity', 'tmdb_score':'TMDB Score'})
df_netflix_movies.head() # rename para un mejor entendimiento de las variables

,ID,Title,Type,Description,Release Year,Age Certification,Runtime,Genres,Production Countries,Seasons,IMDB ID,IMDB Score,IMDB Votes,TMDB Popularity,TMDB Score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


In [ ]:
df_netflix_movies.columns # variables ya renombradas del dataset

Index(['ID', 'Title', 'Type', 'Description', 'Release Year',
       'Age Certification', 'Runtime', 'Genres', 'Production Countries',
       'Seasons', 'IMDB ID', 'IMDB Score', 'IMDB Votes', 'TMDB Popularity',
       'TMDB Score'],
      dtype='object')

In [ ]:
df_netflix_movies.info() #resumen del dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5850 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    5850 non-null   object 
 1   Title                 5849 non-null   object 
 2   Type                  5850 non-null   object 
 3   Description           5832 non-null   object 
 4   Release Year          5850 non-null   int64  
 5   Age Certification     3231 non-null   object 
 6   Runtime               5850 non-null   int64  
 7   Genres                5850 non-null   object 
 8   Production Countries  5850 non-null   object 
 9   Seasons               2106 non-null   float64
 10  IMDB ID               5447 non-null   object 
 11  IMDB Score            5368 non-null   float64
 12  IMDB Votes            5352 non-null   float64
 13  TMDB Popularity       5759 non-null   float64
 14  TMDB Score            5539 non-null   float64
dtypes: float64(5), int64(

In [ ]:
df_netflix_movies.describe()

,Release Year,Runtime,Seasons,IMDB Score,IMDB Votes,TMDB Popularity,TMDB Score
count,5850.000000,5850.000000,2106.000000,5368.000000,5.352000e+03,5759.000000,5539.000000
mean,2016.417094,76.888889,2.162868,6.510861,2.343938e+04,22.637925,6.829175
std,6.937726,39.002509,2.689041,1.163826,9.582047e+04,81.680263,1.170391
min,1945.000000,0.000000,1.000000,1.500000,5.000000e+00,0.009442,0.500000
25%,2016.000000,44.000000,1.000000,5.800000,5.167500e+02,2.728500,6.100000
50%,2018.000000,83.000000,1.000000,6.600000,2.233500e+03,6.821000,6.900000
75%,2020.000000,104.000000,2.000000,7.300000,9.494000e+03,16.590000,7.537500
max,2022.000000,240.000000,42.000000,9.600000,2.294231e+06,2274.044000,10.000000
